# Iterative Quantum Phase Estimation (IQPE)

In notebook **03_AmplitudeAmplification_Procedure** the **Amplification Amplitude** procedure for calculating $E_{x\sim p}(f)$ was reviewed. As showed, in order to take advantage of this procedure, a **Quantum Phase Estimation** (**QPE**) algorithm that relies in the inverse of the **Quantum Fourier Transform** (**QFT**) is needed. Quantum circuit implementation of **QFT** are very complex and very long and deep so its use in actual quantum computers is noisy and not very useful.  

Present notebook reviews an **amplitude amplification** algorithm that does not rely in the inverse of the **Quantum Fourier Transform** (**QFT**): the **Iterative Quantum Phase Estimation** algorithm (**IQPE**).

In order to use this algorithm the following operators are needed:

* $\mathcal{P}$ for loading probability distribution $p(x)$
* $\mathcal{F}$ for loading a function $f(x)$

Present notebook and module are based on the following references:

* *Dobšíček, Miroslav and Johansson, Göran and Shumeiko, Vitaly and Wendin, Göran*. Arbitrary accuracy iterative quantum phase estimation algorithm using a single ancillary qubit: A two-qubit benchmark. Physical Review A 3(76), 2007. https://arxiv.org/abs/quant-ph/0610214

* *Griffiths, Robert B. and Niu, Chi-Sheng*. Semiclassical Fourier Transform for Quantum Computation. Physical Review Letters, 17 (76), 1996. https://arxiv.org/abs/quant-ph/9511007

* *A. Y. Kitaev*. Quantum measurements and the abelian stabilizer problem. Electronic Colloquium on Computational Complexity, 3(3):1–22, 1996. https://arxiv.org/abs/quant-ph/9511026

* *Monz, Thomas and Nigg, Daniel and Martinez, Esteban A. and Brandl, Matthias F. and Schindler, Philipp and Rines, Richard and Wang, Shannon X. and Chuang, Isaac L. and Blatt, Rainer*. Realization of a scalable Shor algorithm. Science 6277 (351). 2016. https://arxiv.org/abs/1507.08852

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic
%matplotlib inline

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        linalg_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        linalg_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    linalg_qpu = PyLinalg() 

In [ ]:
sys.path.append("../libraries/")

## 1. Creating base gate for Grover-like operator

First we need to discretized the probability $p(x)$ and the function $f(x)$.

In [ ]:
from utils import  get_histogram

In [ ]:
#Functions f and p
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

In [ ]:
%matplotlib inline
plt.plot(X, p_X, 'o')
plt.plot(X, f_X, 'o')
plt.legend(['Probability', 'Array'])

Now we create the correspondent operators $\mathcal{P}$ and $\mathcal{F}$ and their correspondent composition.

In [ ]:
from data_loading import load_probability, load_array, load_pf

In [ ]:
p_gate = load_probability(p_X)
f_gate = load_array(np.sqrt(f_X))
pf_gate = load_pf(p_gate, f_gate)

In [ ]:
#circuit = Qprog.to_circ()
%qatdisplay pf_gate

## 2. Class IQPE: algorithm step by step 

The main problem is the following: We have an **Groover** like operator, $\mathcal{Q}$ which is equivalent to a rotation around **y-axis** of a $\theta$ angle. This angle is unknow (a priori) and we want to compute it. We know that using **QPE** with **QFT** allows us get the angle but we know too that **QFT** is a complex and a prone error operation for a quantum computer. Using **IQPE** this $\theta$ can be obtained without usign $\mathcal{QFT}$.

We have implemented and python class called **IQPE** (in the script **iterative_quantum_pe.py**) that allows us implement the **IQPE** algorithm. In this section we are going to describe the class step by step and explain the basics of the **IQPE** algorithm


### Calling the **IQPE** class

In [ ]:
#Load Class
from iterative_quantum_pe import IterativeQuantumPE

In order to instantiate the class we need to provide a pyhton dictionary. Most important keys are:

* oracle : QLM gate or routine with the oracle needed for creating correspondient Grover-like operator
* initial_state : QLM Program with an initial state $|\Psi\rangle$ was loaded. 
* grover :  QLM gate or routine with a Grover-like operator $\mathcal{Q}$ ready for be applied to initial state $|\Psi\rangle$.

If the user provide an *oracle* key then keys *initial_state* and *grover* will be not used. If *oracle* is not provide then user should provide *initial_state* and *grover*.

Other important keys are:

* cbits_number : int with the number of classical bits needed for for phase estimation
* qpu : QLM solver
* shots : int number of shots for quantum job. If 0 exact probabilities will be computed


By instantiate the **IterativeQuantumPE** the class create following properties:

* *init_q_prog*: this property stores the QLM program with the initial state $|\Psi\rangle$.
    * If *oracle* was provided class creates the program using the oracle
    * If *oracle* was **NOT** provided this will be the *initial_state* key
* *q_gate*: this propety stores the correspondient Grover-like operator $\mathcal{Q}$.
    * If *oracle* was provided class creates the Grover-like operator using the oracle
    * If *oracle* was **NOT** provided this will be the *grover* key
    

### Giving and Oracle

In [ ]:
#Giving an Oracle
n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict_0 = {
    'oracle': pf_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    #'easy': True,
    #'easy': False    
}
#Instanciate the class
IQPE_0 = IterativeQuantumPE(**iqpe_dict_0)

In [ ]:
print('QLM loading program')
circuit = IQPE_0.init_q_prog.to_circ()
%qatdisplay circuit --depth 1
print('Grover-like Operator')
q_gate = IQPE_0.q_gate
%qatdisplay q_gate --depth 1

### Giving $|\Psi\rangle$ and $\mathcal{Q}$

In [ ]:
#Given Initial State and Grover operator
from data_extracting import create_qprogram
initial_state = create_qprogram(pf_gate)
from amplitude_amplification import load_q_gate
grover = load_q_gate(pf_gate)

n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict_1 = {
    'initial_state': initial_state,
    'grover': grover,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    #'easy': True,
    #'easy': False    
}
IQPE_1 = IterativeQuantumPE(**iqpe_dict_1)

In [ ]:
print('QLM loading program')
circuit = IQPE_1.init_q_prog.to_circ()
%qatdisplay circuit --depth 1
print('Grover-like Operator')
q_gate = IQPE_1.q_gate
%qatdisplay q_gate --depth 1

###  BE AWARE!!!

In [ ]:
#If oracle not provide or is None and initial_state or grover were not provide an Exception is raised
n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict_2 = {
    #'oracle': None,
    #'initial_state': None,
    #'grover': grover,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    #'easy': True,
    #'easy': False    
}
IQPE_2 = IterativeQuantumPE(**iqpe_dict_2)

The order for execute a **IQPE** algorithm is the following.

#### 1. Initialize the quantum program.

The first thing is created a deep copy of the *init_q_prog* property on the *q_prog* property

In [ ]:
#Initialize the quantum program
IQPE_0.init_iqpe()
IQPE_1.init_iqpe()

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
#Additionally a auxiliar qbit bits was allocated
circuit = IQPE_0.q_prog.to_circ(submatrices_only=True)

%qatdisplay circuit --depth 1

In [ ]:
#Now we have the initial quantum program stored in the property q_prog
#Additionally a auxiliar qbit bits was allocated
circuit = IQPE_1.q_prog.to_circ(submatrices_only=True)

%qatdisplay circuit --depth 1

#### 2. Applying IQPE algorithm

Second thing is apply the IQPE algorithm to the quantum program stored in the *q_prog* property.

In [ ]:
#Execute IQPE algorithm
IQPE_0.apply_iqpe()
IQPE_1.apply_iqpe()

#### 3. Create the QLM circuit

We need to create the quantum circuit of the correspondient quantum program with the complete **IQPE** algorithm using the get_circuit() method

In [ ]:
#Class have a method to create a quantum circuit from quantum program
IQPE_0.get_circuit()
circuit = IQPE_0.circuit
%qatdisplay circuit  --depth 0

In [ ]:
#Class have a method to create a quantum circuit from quantum program
IQPE_1.get_circuit()
circuit = IQPE_1.circuit
%qatdisplay circuit  --depth 0

#### 4. Create the QLM job

We need to create the QLM job from the QLM circuit using the *get_job()* method

In [ ]:
#Class have a method to generate a job from the circuit created in the previous cell
IQPE_0.get_job()
IQPE_1.get_job()

#### 5. Submiting job

With the method *get_job_result()* the job is submitted and the results of the simulation are obtained

In [ ]:
#There is a method for executing a job
IQPE_0.get_job_result()
IQPE_1.get_job_result()

In [ ]:
#Property job_result stores the results of the execution of the job
print(IQPE_0.job_result)

In [ ]:
print(IQPE_1.job_result)

#### 6. Post-Processing the simulation results

Final some postprocesing of the simulated results is needed in order to get the results in a propper way: *get_classicalbits()* method is used. The property **results** store the results in a pandas Dataframe

In [ ]:
#Finally we want to get the results in a straigtoforward way: we use method get_classicalbits
IQPE_0.get_classicalbits()
#In property results we store the results as a pandas DataFrame
IQPE_0.results

In [ ]:
#Finally we want to get the results in a straigtoforward way: we use method get_classicalbits
IQPE_1.get_classicalbits()
#In property results we store the results as a pandas DataFrame
IQPE_1.results

The **property** results is a pandas DataFrame with the following columns:

* **BitString**: is the result of the clasical bits measurement in each step of the algorithm
* **BitInt**: integer representation of the **BitString**
* **Phi**: is the estimated obtained phase and it is computed as: $\frac{BitInt}{2^{c_b}}$ where $c_b$ is the number of classical bits 

In order to obtain the results for classical **amplitude amplification** problem the **proccess_output()** methdo can be used. This method creates a property called **final_results** where final results are stored.

In [ ]:
IQPE_0.proccess_output()
IQPE_0.final_results

In [ ]:
IQPE_1.proccess_output()
IQPE_1.final_results

In [ ]:
np.sum(p_X*f_X)

**final_results** property is the **results** property with some columns with useful calculated values:

* **Theta Unitary**: is the phase egienvalue of the Grover-like operator (2*$\pi$*Phi).
* **Theta**: is the rotation angle $\theta$ applied for the Grover-like operator ($\pi$*Phi)
* **E_p(f)**: ius the desired $E_{x\sim p}(f)$

## 3. Class IQPE: complete execution

In [ ]:
#We create a python dictionary for configuration of class
n_cbits = 6
#We create a python dictionary for configuration of class
iqpe_dict = {
    'oracle': pf_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    #'easy': True,
    'easy': False    
}

In [ ]:
#Create the object
iqpe_ = IterativeQuantumPE(**iqpe_dict)
#Execute complete algorithm
iqpe_.iqpe()

In [ ]:
iqpe_.results

In [ ]:
iqpe_.final_results

### Multiple executions

In [ ]:
#We can do several circuit executions configuring input dictionary properly
iqpe_dict = {
    'oracle': pf_gate,
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    'easy': False,
    'shots': 100
}
iqpe_ = IterativeQuantumPE(**iqpe_dict)
iqpe_.iqpe()

In [ ]:
iqpe_.final_results

In [ ]:
iqpe_.final_results.sort_values('Theta', inplace=True)
plt.plot(iqpe_.final_results['Theta'], iqpe_.final_results['Probability'], 'o')

In [ ]:
iqpe_.final_results.sort_values('theta_90', inplace=True)
plt.plot(iqpe_.final_results['theta_90'], iqpe_.final_results['Probability'], 'o')

In [ ]:
plt.hist(iqpe_.final_results['theta_90'])

In [ ]:
plt.hist(iqpe_.final_results['E_p(f)'])

In [ ]:
iqpe_.final_results['E_p(f)'].describe()

In [ ]:
sum(p_X*f_X)

## 4. Qiskit Test

Present section explains how to implement the **IQPE** example from Qiskit textbook using our library class. Following links have the Qiskit examples:

https://qiskit.org/textbook/ch-labs/Lab04_IterativePhaseEstimation.html

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb


In [ ]:
#Number Of Qbits
n_qbits = 1
#Number Of Classical Bits
n_cbits = 2

In the Qiskit example they try to estimate the phase for 

![title](Qiskit_IQPE.png)

In [ ]:
from qat.lang.AQASM import X, PH, QRoutine

In [ ]:
#Basic Initial circuit and unitary operator whose autovalue we want to compute
#Initial_State Program
from qat.lang.AQASM import Program, H, X, PH
initial_state = Program()
q_bits = initial_state.qalloc(n_qbits)
for i in range(n_qbits):
    initial_state.apply(X, q_bits[i])
grover = PH(np.pi/2.0)

In [ ]:
c = initial_state.to_circ()
%qatdisplay c
%qatdisplay grover

In [ ]:
from iterative_quantum_pe import IterativeQuantumPE

In [ ]:
iqpe_dict = {
    'initial_state': initial_state,
    'grover': grover,    
    'qpu' : linalg_qpu,
    'cbits_number' : n_cbits,
    'shots': 1000,
    #'easy': True
    'easy': False
}
IQPE = IterativeQuantumPE(**iqpe_dict)
IQPE.iqpe()

In [ ]:
easy_circuit = IQPE.circuit
%qatdisplay easy_circuit  --depth 0

In [ ]:
IQPE.results

In [ ]:
plt.hist(IQPE.results['Phi'])

In [ ]:
IQPE.results['Phi'].describe()

As can be seen in 

https://github.com/Qiskit/qiskit-tutorials/blob/master/tutorials/algorithms/09_IQPE.ipynb 

solution in qiskit is just 0.25 for the before configuration